In [77]:
from yfinance import Ticker

ticker = Ticker("TSLA")

# Get stock info
info = ticker.info

# Get historical market data
hist = ticker.history()

# Rename the index from timestamp to date
hist.index = hist.index.strftime("%Y-%m-%d")

# hist.index become a new column
hist.reset_index(inplace=True)

# # Round each row from float to 2 decimal places
hist = hist.round(2)

print(hist)

hist = hist[["Date", "Close", "Volume"]]

hist['Volume'] = (hist['Volume'] / 1000000).round(2)

# # Get each day price change in percentage
hist["Change"] = (hist["Close"].pct_change() * 100).round(2)

hist.columns = ['Date(utc-4)', 'Close', 'Volume(M)', 'Change(%)']

hist

          Date    Open    High     Low   Close     Volume  Dividends  \
0   2024-04-25  158.96  170.88  158.36  170.18  126427500        0.0   
1   2024-04-26  168.85  172.12  166.37  168.29  109815700        0.0   
2   2024-04-29  188.42  198.87  184.54  194.05  243869700        0.0   
3   2024-04-30  186.98  190.95  182.84  183.28  127031800        0.0   
4   2024-05-01  182.00  185.86  179.01  179.99   92829700        0.0   
5   2024-05-02  182.86  184.60  176.02  180.01   89148000        0.0   
6   2024-05-03  182.10  184.78  178.42  181.19   75491500        0.0   
7   2024-05-06  183.80  187.56  182.20  184.76   84390300        0.0   
8   2024-05-07  182.40  183.26  177.40  177.81   75045900        0.0   
9   2024-05-08  171.59  176.06  170.15  174.72   79969500        0.0   
10  2024-05-09  175.01  175.62  171.37  171.97   65950300        0.0   
11  2024-05-10  173.05  173.06  167.75  168.47   72627200        0.0   
12  2024-05-13  170.00  175.40  169.00  171.89   67018900       

,Date(utc-4),Close,Volume(M),Change(%)
0,2024-04-25,170.18,126.43,NaN
1,2024-04-26,168.29,109.82,-1.11
2,2024-04-29,194.05,243.87,15.31
3,2024-04-30,183.28,127.03,-5.55
4,2024-05-01,179.99,92.83,-1.80
5,2024-05-02,180.01,89.15,0.01
6,2024-05-03,181.19,75.49,0.66
7,2024-05-06,184.76,84.39,1.97
8,2024-05-07,177.81,75.05,-3.76
9,2024-05-08,174.72,79.97,-1.74


# IEX Cloud

In [75]:
import requests
import pandas as pd

# 輸入你的 IEX Cloud API Key
api_token = 'YOUR_IEX_CLOUD_API_TOKEN'

# 定義函數來獲取 IEX Cloud 的公司概覽數據
def get_iex_cloud_overview(symbol, api_token):
    url = f'https://cloud.iexapis.com/stable/stock/{symbol}/company?token={api_token}'
    response = requests.get(url)
    return response.json()

# 定義函數來獲取 IEX Cloud 的財務數據
def get_iex_cloud_financials(symbol, api_token):
    url = f'https://cloud.iexapis.com/stable/stock/{symbol}/financials?token={api_token}'
    response = requests.get(url)
    return response.json()

# 定義函數來獲取 IEX Cloud 的最新股價數據
def get_iex_cloud_quote(symbol, api_token):
    url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={api_token}'
    response = requests.get(url)
    return response.json()

# 獲取Google的公司概覽、財務數據和最新股價數據
symbol = 'GOOGL'
overview_data = get_iex_cloud_overview(symbol, api_token)
financials_data = get_iex_cloud_financials(symbol, api_token)
quote_data = get_iex_cloud_quote(symbol, api_token)

# 提取必要數據
market_cap = float(overview_data['marketCap'])
financial_data = financials_data['financials'][0]  # 最新的財務報告
total_debt = float(financial_data['totalDebt'])
shares_outstanding = float(quote_data['sharesOutstanding'])
cost_of_equity = float(overview_data['returnOnEquity']) / 100
cost_of_debt = float(financial_data['interestExpense']) / total_debt
tax_rate = float(financial_data['incomeTax']) / float(financial_data['grossProfit'])  # 這是近似值
net_debt = total_debt - float(financial_data['cash'])

# 提取未來的現金流量（這裡假設只取最近一年的數據並進行推算）
future_cash_flows = [
    float(financial_data['operatingCashFlow']) / 1e9,  # 轉換為十億美元
    float(financial_data['operatingCashFlow']) * 1.1 / 1e9,
    float(financial_data['operatingCashFlow']) * 1.2 / 1e9,
    float(financial_data['operatingCashFlow']) * 1.3 / 1e9,
    float(financial_data['operatingCashFlow']) * 1.4 / 1e9
]

# 假設數據
perpetual_growth_rate = 0.03

# 計算WACC
equity_value = market_cap
debt_value = total_debt
WACC = (equity_value / (equity_value + debt_value)) * cost_of_equity + \
       (debt_value / (equity_value + debt_value)) * cost_of_debt * (1 - tax_rate)

print(f"Google (GOOGL) 的加權平均資本成本 (WACC): {WACC:.2%}")

# 取得最新的收盤價
latest_close_price = quote_data['latestPrice']
print(f"Google (GOOGL) 的最新收盤價: {latest_close_price} 美元")

# 計算每年的折現現值
discounted_cash_flows = [fcf / (1 + WACC)**t for t, fcf in enumerate(future_cash_flows, start=1)]

# 計算終值
terminal_value = (future_cash_flows[-1] * (1 + perpetual_growth_rate)) / (WACC - perpetual_growth_rate)
discounted_terminal_value = terminal_value / (1 + WACC)**len(future_cash_flows)

# 計算企業價值
enterprise_value = sum(discounted_cash_flows) + discounted_terminal_value

# 計算股權價值和每股價值
equity_value = enterprise_value - net_debt
per_share_value = equity_value / shares_outstanding

# 用DataFrame展示計算過程
df = pd.DataFrame({
    'Year': range(1, 6),
    'Future Cash Flow (Billion USD)': future_cash_flows,
    'Discounted Cash Flow (Billion USD)': discounted_cash_flows
})
df.loc[len(df)] = ['Terminal Value', terminal_value, discounted_terminal_value]
df.loc[len(df)] = ['Enterprise Value', '', enterprise_value]
df.loc[len(df)] = ['Net Debt', '', net_debt]
df.loc[len(df)] = ['Equity Value', '', equity_value]
df.loc[len(df)] = ['Shares Outstanding (Billion)', '', shares_outstanding]
df.loc[len(df)] = ['Per Share Value (USD)', '', per_share_value]

print(df)
print(f"每股價值: {per_share_value} 美元")
print(f"Google (GOOGL) 的最新收盤價: {latest_close_price} 美元")


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Alpha Vantage

In [76]:
import requests
import yfinance as yf
import pandas as pd

# 輸入你的 Alpha Vantage API Key
api_key = 'YOUR_ALPHA_VANTAGE_API_KEY'

# 定義函數來獲取Alpha Vantage的公司概覽數據
def get_alpha_vantage_overview(symbol, api_key):
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    return response.json()

# 定義函數來獲取Alpha Vantage的現金流量數據
def get_alpha_vantage_cashflow(symbol, api_key):
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    return response.json()

# 獲取Google的公司概覽和現金流量數據
symbol = 'GOOGL'
overview_data = get_alpha_vantage_overview(symbol, api_key)
cashflow_data = get_alpha_vantage_cashflow(symbol, api_key)

# 提取必要數據
market_cap = float(overview_data['MarketCapitalization'])
total_debt = float(overview_data['TotalDebt'])
shares_outstanding = float(overview_data['SharesOutstanding'])
cost_of_equity = float(overview_data['ReturnOnEquity']) / 100
cost_of_debt = float(overview_data['InterestExpense']) / float(overview_data['TotalDebt'])
tax_rate = float(overview_data['EffectiveTaxRate']) / 100
net_debt = total_debt - float(overview_data['CashAndCashEquivalents'])

# 提取未來的現金流量（這裡假設只取最近一年的數據並進行推算）
future_cash_flows = [
    float(cashflow_data['annualReports'][0]['operatingCashflow']) / 1e9,  # 轉換為十億美元
    float(cashflow_data['annualReports'][0]['operatingCashflow']) * 1.1 / 1e9,
    float(cashflow_data['annualReports'][0]['operatingCashflow']) * 1.2 / 1e9,
    float(cashflow_data['annualReports'][0]['operatingCashflow']) * 1.3 / 1e9,
    float(cashflow_data['annualReports'][0]['operatingCashflow']) * 1.4 / 1e9
]

# 假設數據
perpetual_growth_rate = 0.03

# 計算WACC
equity_value = market_cap
debt_value = total_debt
WACC = (equity_value / (equity_value + debt_value)) * cost_of_equity + \
       (debt_value / (equity_value + debt_value)) * cost_of_debt * (1 - tax_rate)

print(f"Google (GOOGL) 的加權平均資本成本 (WACC): {WACC:.2%}")

# 下載Google (GOOGL) 的股票數據
googl = yf.Ticker(symbol)
googl_info = googl.history(period="1d")

# 取得最新的收盤價
latest_close_price = googl_info['Close'].iloc[0]
print(f"Google (GOOGL) 的最新收盤價: {latest_close_price} 美元")

# 計算每年的折現現值
discounted_cash_flows = [fcf / (1 + WACC)**t for t, fcf in enumerate(future_cash_flows, start=1)]

# 計算終值
terminal_value = (future_cash_flows[-1] * (1 + perpetual_growth_rate)) / (WACC - perpetual_growth_rate)
discounted_terminal_value = terminal_value / (1 + WACC)**len(future_cash_flows)

# 計算企業價值
enterprise_value = sum(discounted_cash_flows) + discounted_terminal_value

# 計算股權價值和每股價值
equity_value = enterprise_value - net_debt
per_share_value = equity_value / shares_outstanding

# 用DataFrame展示計算過程
df = pd.DataFrame({
    'Year': range(1, 6),
    'Future Cash Flow (Billion USD)': future_cash_flows,
    'Discounted Cash Flow (Billion USD)': discounted_cash_flows
})
df.loc[len(df)] = ['Terminal Value', terminal_value, discounted_terminal_value]
df.loc[len(df)] = ['Enterprise Value', '', enterprise_value]
df.loc[len(df)] = ['Net Debt', '', net_debt]
df.loc[len(df)] = ['Equity Value', '', equity_value]
df.loc[len(df)] = ['Shares Outstanding (Billion)', '', shares_outstanding]
df.loc[len(df)] = ['Per Share Value (USD)', '', per_share_value]

print(df)
print(f"每股價值: {per_share_value} 美元")
print(f"Google (GOOGL) 的最新收盤價: {latest_close_price} 美元")

KeyError: 'TotalDebt'

In [84]:
# Discounted Cashflow Model
def Discounted_Cashflow_Model(cash_flow, year, growth_rate, discount_rate, growth_rate_forever, WACC, cash_and_short_term_investment, net_debt, shares_outstanding):
    # 終值 = 預測期最後一年的現金流 × ( 1 + 永久增長率 ) / ( 折現率 − 永久增長率 )
    final_value = (cash_flow * (1 + growth_rate)**year ) * (1 + growth_rate_forever) / (discount_rate - growth_rate_forever)

    # 企業價值 = 現值 = ∑( 現金流 × ( 1 + 成長率 )^n / ( 1 + WACC )^n ) + 終值 / ( 1 + WACC )^n
    present_value = sum(cash_flow * ( 1 + growth_rate ) ** y / ( 1 + WACC ) ** y for y in range(1, year + 1)) + final_value / (1 + WACC) ** year

    # 股權價值 = 企業價值 + 現金及短期頭資 − 淨債務
    equity_value = present_value + cash_and_short_term_investment - net_debt

    # 每股價值(合理價) = 股權價值 / 流通在外股數
    per_share_value = equity_value / shares_outstanding

    return per_share_value

# 現金流
cash_flow = 100
# 預測未來五年
year = 5
# 未來n年成長率 10%
growth_rate = 0.1
# 折現率 8%
discount_rate = 0.08
# 永久增長率 2.5%
growth_rate_forever = 0.025
# 加權平均資本成本 (WACC)
WACC = 0.08
# 現金及短期頭資
cash_and_short_term_investment = 500
# 淨債務
net_debt = 800
# 流通在外股數
shares_outstanding = 100

dcf = Discounted_Cashflow_Model(cash_flow, year, growth_rate, discount_rate, growth_rate_forever, WACC, cash_and_short_term_investment, net_debt, shares_outstanding)
dcf

22.711790560946557